In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
proxy_host = "127.0.0.1"
proxy_port = "9090"
proxy_auth = ":"
proxies = {
       "https": "https://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port),
       "http": "http://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port)
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-GPC': '1',
    'If-Modified-Since': 'Friday, 03 December 2021 03:30:27',
    'Cache-Control': 'max-age=0',
}

In [3]:
ousama_ranking = "https://readmanganato.com/manga-mo989871"
one_piece = "https://www.readm.org/manga/5404"

In [4]:
response = requests.get(ousama_ranking, headers=headers, ) #  proxies=proxies
if response.status_code != 200:
    raise Exception(f"request error{r.status_code}")

In [5]:
soup = BeautifulSoup(response.content, features="lxml")

In [6]:
chapters = [chapter["href"] for chapter in soup.findAll("a", {"class": "chapter-name text-nowrap"})]

In [7]:
chapters[0]

'https://readmanganato.com/manga-mo989871/chapter-23'

In [8]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [ ]:
driver.get(chapters[0])

In [9]:
list_of_img = driver.find_elements_by_tag_name('img')

In [14]:
selenium_imgs = [l for l in list_of_img if "chapter" in l.get_attribute("src")]

In [15]:
print(len(selenium_imgs))

15


In [30]:
def sace_to_pdf(images, filen_name):
    images[0].save(
        f"{filen_name}.pdf",
        "PDF",
        resolution=100.0,
        save_all=True,
        append_images=images[1:],
    )

In [21]:
from PIL import Image
from io import BytesIO

In [22]:
# for img in selenium_imgs:
#     image_url = img.get_attribute("src")
#     chapter_name = os.path.basename(os.path.dirname(image_url))
#     file_name = os.path.basename(image_url)
#     with open(f"{chapter_name}-{file_name}", 'wb') as file:
#         file.write(img.screenshot_as_png)

In [23]:
images = [Image.open(BytesIO(image.screenshot_as_png)).convert('RGB') for image in selenium_imgs]

In [26]:
import os

In [27]:
image_url = selenium_imgs[0].get_attribute("src")
chapter_name = os.path.basename(os.path.dirname(image_url))

In [28]:
chapter_name

'vol2_chapter_23'

In [31]:
sace_to_pdf(images, f"{chapter_name}-Original")

In [37]:
import numpy as np
# https://answers.opencv.org/question/219194/black-and-white-image-detection/
def detect_color_image(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv_img)   
    
    return len(np.unique(s))>5

In [44]:
def convert_to_dark_mode(images, mode):
    
    new_images = []
    for image in images:
        if detect_color_image(np.array(image)):
            new_images.append(image)
        else:
            cv2_image = np.array(image)

            if mode < 0:
                new_image = cv2.bitwise_not(cv2_image)
            else:
                new_image = cv2.applyColorMap(cv2.bitwise_not(cv2_image), mode)

            new_images.append(Image.fromarray(new_image))    
    return new_images

In [49]:
sace_to_pdf(convert_to_dark_mode(images, -1), f"{chapter_name}-bitwise_not")

In [ ]:
# COLORMAP_PINK
# COLORMAP_BONE
# COLORMAP_DEEPGREEN
# cv2.COLORMAP_HOT